### Data Preparation

In [ ]:
import sys
sys.path.append('/Users/broniy/Desktop/CreativeRank/')
%load_ext autoreload
%autoreload 2

import os 
import mlflow
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.model_selection import ParameterGrid
import numpy as np
from catboost import CatBoostClassifier, CatBoostRanker, Pool
from typing import List
from settings import DATA_FOLDER

In [ ]:
clicked_df = pd.read_csv(
    DATA_FOLDER / "processed/clicked.csv"
)
non_clicked_df = pd.read_csv(
    DATA_FOLDER / "processed/non_clicked_large.csv"
)

variations_df = pd.read_csv(
    DATA_FOLDER / "processed/feats_df.csv"
).rename(columns={"id": "VARIATION_ID"}).fillna("UNK")
variations_df = variations_df[~variations_df['error'].isna()].drop(columns=['error'])



In [ ]:
users_df = (
    pd.concat([clicked_df, non_clicked_df], axis=0)
    .assign(
        CLICK=lambda x: (x["CLICK_COUNT"] > 0).astype(int),
        EXPERIMENT_DATE=lambda x: pd.to_datetime(
            {
                "year": 2025,
                "month": x["MONTH"],
                "day": x["DAY"],
            }
        ),
    )
    .dropna(subset=["CLICK_COUNT"])
    .drop(columns=["RN"])
    .fillna(
        value={
            "TOTAL_ORDERS_VALUE": 0,
            "AVG_ORDER_VALUE": 0,
            "LAST_ORDER_VALUE": 0,
            "COUNTRY": "UNK",
            "REGION": "UNK",
            "LATEST_CLICK_CLIENT_TYPE": "UNK",
            "LATEST_CLICK_CLIENT_NAME": "UNK",
            "LATEST_CLICK_CLIENT_OS_FAMILY": "UNK",
            "FIRST_UTM_SOURCE": "UNK",
            "FIRST_UTM_CONTENT": "UNK",
            "FIRST_UTM_CAMPAIGN": "UNK",
            "LAST_UTM_SOURCE": 'UNK', "LAST_UTM_CONTENT": 'UNK', "LAST_UTM_CAMPAIGN": 'UNK',
            "CITY": "UNK",
            "TIMEZONE": "UNK",
        }
    )
)
# Convert FIRST_ACTIVE_TS to datetime
users_df["FIRST_ACTIVE_TS_dt"] = pd.to_datetime(users_df["FIRST_ACTIVE_TS"])

# Compute months between today and FIRST_ACTIVE_TS
today = pd.Timestamp(datetime.today())

# Compute years and months difference and convert to total months
users_df["MONTHS_SINCE_FIRST_ACTIVE"] = (
    today.year - users_df["FIRST_ACTIVE_TS_dt"].dt.year
) * 12 + (today.month - users_df["FIRST_ACTIVE_TS_dt"].dt.month)


users_df = users_df[users_df["VARIATION_ID"].isin(variations_df["VARIATION_ID"])]
users_df = users_df.drop_duplicates()

# Print the size of users_df before removal
print(f"users_df size before removing small experiments: {users_df.shape[0]} rows")
# Remove experiments with less than 100 participants
experiment_counts = users_df.groupby("EXPERIMENT_ID")["RECIPIENT_ID"].nunique()
valid_experiments = experiment_counts[experiment_counts >= 100].index
users_df = users_df[users_df["EXPERIMENT_ID"].isin(valid_experiments)]
# Print the size of users_df after removal
print(f"users_df size after removing small experiments: {users_df.shape[0]} rows")


users_df size before removing small experiments: 501008 rows
users_df size after removing small experiments: 500953 rows


In [ ]:
users_df.groupby(["EXPERIMENT_ID", "EXPERIMENT_DATE"]).agg({"CLICK": ["sum", "count"]})

CLICK       
                                                       sum  count
EXPERIMENT_ID                        EXPERIMENT_DATE             
002deaf7-331f-4b5e-866b-f6dad60e4a79 2025-07-28       1355  14905
00bb26ff-6fe3-4465-ac77-12bfc33aa6df 2025-07-17       1787  19657
0ef6d2e9-7601-4df6-a215-83e6e79aa24e 2025-10-06       1293  14223
11c49e5d-21ac-4d6d-88c3-f211562a8e07 2025-09-17       1156  12647
1d6dbba7-dcc5-46f4-a4aa-aef3124a8fcf 2025-09-05       1959  21549
2a3f341e-1807-4eb3-9d8d-202c32d52632 2025-08-25       1075  11825
3ecf34fc-1f15-4b32-970f-4061544da763 2025-07-14       1835  20179
43d750b5-8698-4cf0-9ea2-f705f4f196ed 2025-09-25       1968  21585
44d26695-cdf2-41a4-b161-393fdaf964bc 2025-07-26       2122  23342
49c33d7c-ef04-43a7-bbd0-783489c64849 2025-09-06       1757  19327
6a258715-721a-41e9-8abb-af41308c1f48 2025-08-19       2051  22561
6f506df9-be60-452d-b914-8230c29c2ff1 2025-07-22       1622  17838
78a802ae-d6cd-4f39-aecb-138668fa2607 2025-10-02       1139  12529
81ae4870-e57d-4bc4-a2d7-48ffa5411707 2025-07-10       2113  21852
823158da-7b0a-4c19-8189-663c22a3ae38 2025-09-27       2286  25146
8ea67496-0fb3-4efd-8cea-4b8d88351b8e 2025-07-01       1766  11144
91eee220-fee7-488b-952a-c96aa8e493db 2025-08-14       1627  17897
9cd54b2b-31f9-43e4-9073-0d2b61bf9f15 2025-08-30       2039  22429
9defe9fd-0374-4de6-99f7-aaa392903d67 2025-08-12       2244  24684
a1db5d4b-b641-4d3d-a8a6-d4c9bdd9c033 2025-08-23       1543  16973
c5288ca2-3928-4364-8f08-bebc1036dd87 2025-07-11       2626  28801
cd4a656f-290a-41e2-be1d-bf62ad85757d 2025-09-29       2186  23998
e4b4a349-3b14-439e-946f-f716101dac69 2025-08-02        853   9356
e627d7f0-46c4-4894-872e-59a2fc108c30 2025-08-07        356   3579
e697ab50-0abb-42d3-92a0-43f1ed597476 2025-08-29       3678  31476
f93bf2bd-1d50-4131-9ec2-223a4d9987e8 2025-09-23       2921  31451

In [ ]:
USER_COLS = [
    "RECIPIENT_ID",
    "COUNTRY",
    "REGION",
    "LATEST_CLICK_CLIENT_TYPE",
    "LATEST_CLICK_CLIENT_NAME",
    "LATEST_CLICK_CLIENT_OS_FAMILY",
    "TOTAL_ORDERS_VALUE",
    "AVG_ORDER_VALUE",
    "LAST_ORDER_VALUE",
    "MONTHS_SINCE_FIRST_ACTIVE",
    "CLICK",
    "FIRST_UTM_SOURCE", "FIRST_UTM_CONTENT", "FIRST_UTM_CAMPAIGN",
    'LAST_UTM_SOURCE', 'LAST_UTM_CONTENT', 'LAST_UTM_CAMPAIGN',
    "CITY",
    "TIMEZONE"
]
VARIATION_COLS = [
   'Q1_CREATIVE', 'Q2_CREATIVE', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q1_SBL', 'Q2_SBL' 
]

CATEGORICAL_COLS = [
    "COUNTRY",
    "REGION",
    "CITY",
    "TIMEZONE",
    "LATEST_CLICK_CLIENT_TYPE",
    "LATEST_CLICK_CLIENT_NAME",
    "LATEST_CLICK_CLIENT_OS_FAMILY",
    "FIRST_UTM_SOURCE", "FIRST_UTM_CONTENT", "FIRST_UTM_CAMPAIGN",
    'LAST_UTM_SOURCE', 'LAST_UTM_CONTENT', 'LAST_UTM_CAMPAIGN',
    'Q1_CREATIVE', 'Q2_CREATIVE', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q1_SBL', 'Q2_SBL' 
]

NUMERICAL_COLS = [
    "TOTAL_ORDERS_VALUE",
    "AVG_ORDER_VALUE",
    "LAST_ORDER_VALUE",
    "MONTHS_SINCE_FIRST_ACTIVE"
]
COLS = CATEGORICAL_COLS + NUMERICAL_COLS

In [ ]:
### Classification results
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

# Ensure experiment_date is datetime
users_df["EXPERIMENT_DATE"] = pd.to_datetime(users_df["EXPERIMENT_DATE"])

# Sort unique experiments by date
experiment_order = (
    users_df[["EXPERIMENT_ID", "EXPERIMENT_DATE"]]
    .sort_values("EXPERIMENT_DATE")
    .drop_duplicates()
    .reset_index(drop=True)
)
# Get last two for test, others for train
test_experiments = experiment_order.tail(2)["EXPERIMENT_ID"]
train_experiments = experiment_order.iloc[:-2]["EXPERIMENT_ID"]

# Join users_df with variation_df on EXPERIMENT_ID and VARIATION_ID
merged_df = users_df.merge(
    variations_df,
    left_on=["EXPERIMENT_ID", "VARIATION_ID"],
    right_on=["EXPERIMENT_ID", "VARIATION_ID"],
    how="left"
)

# Select rows for train/test
train_df = merged_df[merged_df["EXPERIMENT_ID"].isin(train_experiments)]
test_df= merged_df[merged_df["EXPERIMENT_ID"].isin(test_experiments)]
print(len(train_df), len(test_df))

display(merged_df.head())
X_train = train_df[ COLS]
y_train = train_df[ "CLICK"]
X_test = test_df[COLS]
y_test = test_df[ "CLICK"]

# Identify categorical features indices for CatBoost
cat_features = [X_train.columns.get_loc(col) for col in CATEGORICAL_COLS if col in X_train.columns]

clf = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    cat_features=cat_features,
    random_seed=42,
    verbose=100,
    # early_stopping_rounds=500,
    use_best_model=True
)
clf.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

# If you want to report AUC on test:
from sklearn.metrics import roc_auc_score
y_pred = clf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
print(f"Test AUC: {auc:.4f}")
print(f"Best validation score: {clf.best_score_['validation']['AUC']:.4f}")

train_pool, group_ids, y = get_pooled_dataset(train_df)
print(f'group_ids: {group_ids}')
print(f'y: {y}')
import pandas as pd

# group_ids is assumed to align with y: each group corresponds to a slice of y (e.g., lengths)
# But if group_ids is just a Series (not group sizes), groupby works directly.

# If group_ids is an array of group IDs per row in y:
mean_y_per_group = pd.DataFrame({"group_id": group_ids, "y": y}).groupby("group_id")["y"].mean()
print(mean_y_per_group.unique())

import matplotlib.pyplot as plt

# Show the distribution of y (target)
plt.figure(figsize=(6,4))
plt.hist(y, bins=20, edgecolor='k')
plt.title("Distribution of y")
plt.xlabel("y")
plt.ylabel("Count")
plt.show()

# Show the distribution of mean_y_per_group
plt.figure(figsize=(6,4))
plt.hist(mean_y_per_group, bins=20, edgecolor='k')
plt.title("Distribution of mean_y_per_group")
plt.xlabel("Mean y per group")
plt.ylabel("Count")
plt.show()
